<a href="https://colab.research.google.com/github/parkhy0106/HolyMoly/blob/master/%EB%8B%A8%20%ED%95%9C%20%EC%82%AC%EB%9E%8C%EC%9D%84%20%EC%9C%84%ED%97%8C%20%EC%84%9C%EB%A6%AC%20%EC%8B%9C%EB%AE%AC%EB%A0%88%EC%9D%B4%EC%85%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 설정 및 매개변수 선언



In [1]:
ls

blurred_normalmap.png  raw_img.png   vis_high_contrast.png
depth-from-normals/    sample_data/  vis_low_contrast.png


In [2]:
import cv2
import matplotlib.pyplot as plt
import copy
import numpy as np
from google.colab.patches import cv2_imshow
import json

from PIL import Image as PilImage
import tempfile
from tqdm.auto import tqdm
import warnings

In [3]:
from math import pi, sin, cos, sqrt
DEGREES_TO_RADIANS = pi / 180

In [4]:
test_x, test_y = [], []

Cartesian Plotting
To draw fractals, first we need to be able to draw lines.


In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('bmh')  # Use some nicer default colors

In [6]:
def plot_coords(coords, bare_plot=False):
    # Converts a list of coordinates into
    # lists of X and Y values, respectively.
    X, Y = zip(*coords)
    # Draws the plot.
    test_x = copy.deepcopy(X)
    test_y = copy.deepcopy(Y)
    return X, Y

In [7]:
def branching_to_coords(turtle_program, turn_amount=45):
    saved_states = list()
    state = (0, 0, 90)
    yield (0, 0)

    for command in tqdm(turtle_program):
        x, y, angle = state

        if command.lower() in 'abcdefghij':        # Move forward (matches a-j and A-J)
            state = (x - cos(angle * DEGREES_TO_RADIANS),
                     y + sin(angle * DEGREES_TO_RADIANS),
                     angle)

            if command.islower():                  # Add a break in the line if command matches a-j
                yield (float('nan'), float('nan'))

            yield (state[0], state[1])

        elif command == '+':                       # Turn clockwise
            state = (x, y, angle + turn_amount)

        elif command == '-':                       # Turn counterclockwise
            state = (x, y, angle - turn_amount)

        elif command == '[':                       # Remember current state
            saved_states.append(state)

        elif command == ']':                       # Return to previous state
            state = saved_states.pop()
            yield (float('nan'), float('nan'))
            x, y, _ = state
            yield (x, y)

        # Note: We silently ignore unknown commands

In [8]:
def l_plot(axiom, transformations, iterations=0, angle=45):
    turtle_program = transform_multiple(axiom, transformations, iterations)
    coords = branching_to_coords(turtle_program, angle)
    return plot_coords(coords, bare_plot=False) # bare_plot removes the axis labels

In [9]:
def transform_sequence(sequence, transformations):
    return ''.join(transformations.get(c, c) for c in sequence)

transform_sequence('acab', {'a': 'aba', 'c': 'bb'})

'ababbabab'

In [10]:
def transform_multiple(sequence, transformations, iterations):
    for _ in range(iterations):
        sequence = transform_sequence(sequence, transformations)
    return sequence

print('0:', transform_multiple('abba', {'b': 'bab'}, 0))
print('1:', transform_multiple('abba', {'b': 'bab'}, 1))
print('2:', transform_multiple('abba', {'b': 'bab'}, 2))

0: abba
1: ababbaba
2: ababababbabababa


# 서리 만들기

패턴의 생성 규칙은 다음과 같습니다


*   '+': 시계방향 회전
*   '-': 반시계방향 회전
*   '[': 현재 분기에서 재시작
*   ']': 과거 분기로 돌아가기
*   소문자 입력은 선 긋기 없이 이동
*   대문자 입력은 선 그으며 이동


(레터링이 필요한 이유는 문양마다 회귀를 부여해야해서... 스파게티 코드가 되어버렸어요...)

(그래서 일단은 참치님이 필요로 하시는 문양을 기본으로 입력해뒀어요. 조금씩 수정해보면서 모양을 찾아보세요.)

##프렉탈 구조 생성

In [15]:
#@title 프렉탈 구조 생성 변수
#@markdown ###...........................프렉탈 구조 생성...........................#
#@markdown #
lettering = "F" #@param {type: "string"}
#@markdown 서리를 만들기 위한 프렉탈 생성 코드입니다(참치님이 바라는 프렉탈 방향을 일단 넣어놨습니다)
fractal_pattern = "{'F': '+-FF[+++F][-FF]'}" #@param {type:"string"}
res = eval(fractal_pattern)
#@markdown 프렉탈 차원수(높을수록 더 정교한 무늬를 만들지만 계산이 오래 걸려요)
fractal_dimension = 9 #@param {type:"slider", min:1, max:9, step:1}
#@markdown 회전 각도
angle = 23 #@param {type:"slider", min:1, max:150, step:1}
#@markdown #
#@markdown ##...........................시각화...........................#
#@markdown 가장 두꺼운 부분의 두께(px)
main = 3 #@param {type:"slider", min:1, max:10, step:1}
#@markdown 중간 마디 부분의 두께(px)
sub = 2 #@param {type:"slider", min:1, max:10, step:1}
#@markdown 끝마디 부분의 두께(px)
sting = 1 #@param {type:"slider", min:1, max:10, step:1}

In [ ]:
#@title 생성하기(우클릭 하셔서 저장하시거나, 좌측에 vis_high_constrst, blurred_normalmap 다운로드 하셔도 돼요.)
X, Y = l_plot(lettering, res, fractal_dimension, angle)
fr_size = max(max(X),max(Y))
print(f"이미지 크기 = {int(fr_size*5)}")
fr_min = min(X)
blank = np.full((int(fr_size*5),int(fr_size*5),3), 0, dtype=np.uint8)
frame = copy.deepcopy(blank)
frame2 = copy.deepcopy(blank)

#@title dist map 작성
tmp_x, tmp_y = 0, 0
for x_ind, y_ind in tqdm(zip(X, Y)):
  x_ind = x_ind - fr_min
  try:
    dx = ((int(fr_size*5) - tmp_x, int(fr_size*5) - tmp_y)[0] - (int(fr_size*5) - int(x_ind*5), int(fr_size*5) - int(y_ind*5))[0])
    dy = ((int(fr_size*5) - tmp_x, int(fr_size*5) - tmp_y)[1] - (int(fr_size*5) - int(x_ind*5), int(fr_size*5) - int(y_ind*5))[1])
    if sqrt(dx**2 + dy**2) > 20:
      cv2.line(frame, (int(fr_size*5) - tmp_x, int(fr_size*5) - tmp_y), (int(fr_size*5) - int(x_ind*5), int(fr_size*5) - int(y_ind*5)), (255, 255, 255), thickness=main)
    elif sqrt(dx**2 + dy**2) >10 :
      cv2.line(frame, (int(fr_size*5) - tmp_x, int(fr_size*5) - tmp_y), (int(fr_size*5) - int(x_ind*5), int(fr_size*5) - int(y_ind*5)), (255, 255, 255), thickness=sub)
    cv2.line(frame, (int(fr_size*5) - tmp_x, int(fr_size*5) - tmp_y), (int(fr_size*5) - int(x_ind*5), int(fr_size*5) - int(y_ind*5)), (255, 255, 255), thickness=sting)
    tmp_x = int(x_ind*5)
    tmp_y = int(y_ind*5)
  except:
    pass

cv2.imwrite("raw_img.png",frame)

size = 3
kernel = np.ones((size, size), np.float32) / (size ** 2)
dst = cv2.filter2D(frame, -1, kernel)
cv2.imwrite("blurred_normalmap.png", dst)

grey = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)

import cv2
import numpy as np

def get_surface_normal_by_depth(depth, K=None):
    """
    depth: (h, w) of float, the unit of depth is meter
    K: (3, 3) of float, the depth camere's intrinsic
    """
    K = [[1, 0], [0, 1]] if K is None else K
    fx, fy = K[0][0], K[1][1]

    dz_dv, dz_du = np.gradient(depth)  # u, v mean the pixel coordinate in the image
    # u*depth = fx*x + cx --> du/dx = fx / depth
    du_dx = fx / depth  # x is xyz of camera coordinate
    dv_dy = fy / depth

    dz_dx = dz_du * du_dx
    dz_dy = dz_dv * dv_dy
    # cross-product (1,0,dz_dx)X(0,1,dz_dy) = (-dz_dx, -dz_dy, 1)
    normal_cross = np.dstack((-dz_dx, -dz_dy, np.ones_like(depth)))
    # normalize to unit vector
    normal_unit = normal_cross / np.linalg.norm(normal_cross, axis=2, keepdims=True)
    # set default normal to [0, 0, 1]
    normal_unit[~np.isfinite(normal_unit).all(2)] = [0, 0, 1]
    return normal_unit


vis_normal = lambda normal: np.uint8((normal + 1) / 2 * 255)[..., ::-1]

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    normal1 = get_surface_normal_by_depth(grey)    #  spend time: 60ms

cv2.imwrite("vis_low_contrast.png", vis_normal(normal1))

alpha = 2.0
dst_frame = np.clip((1+alpha)*vis_normal(normal1) - 128*alpha, 0, 255).astype(np.uint8)

cv2.imwrite("vis_high_contrast.png", dst_frame)

#@title Result
cv2_imshow(dst)
cv2_imshow(dst_frame)

  0%|          | 0/6835935 [00:00<?, ?it/s]

이미지 크기 = 6670


0it [00:00, ?it/s]